Author: Soumya Sanjay Kumar

In [1]:
import numpy as np
import re
import qiskit

# Importing standard Qiskit libraries
from qiskit.extensions import UnitaryGate
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, assemble, Aer, IBMQ, execute
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.providers.aer import AerSimulator
from math import pi

In [2]:
import qiskit.tools.jupyter
%qiskit_version_table

/home/soumyassk/anaconda3/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [3]:
def create_circuit(qasm_str):
    gate_list = []
    reg_list = []
    reg_names = []

    gate_idxs = []
    reg_idxs = []
    measure_qub_name=[]
    measure_clas_name=[]
    measure_idxs=[]
    for line in qasm_str.strip().split("\n")[2:]:
        if "->" and "measure" not in line:
            idxs = re.findall(r'\[(\d+)\]', line)
            idxs = list(map(int, idxs))

            gatereg = line.split()[0]
    #         print(gatereg)

            # Registers
            if gatereg in ["qreg","creg"]:
                regtype = gatereg
                reg_list.append(regtype)
                reg_idxs.append(idxs)
                reg_name = line.split(regtype)[1].strip().split("[")[0]
                reg_names.append(reg_name)
    #             print(f"{line} -------- {reg_name}")

            # Gates
            else:
                gatetype = gatereg
                gate_list.append(gatetype)
                gate_idxs.append(idxs)
    #         print(line)
    #         print(f"Gate {gate}, idxs {idxs}")
        # Registers
        else:
            idxs_m = re.findall(r'\[(\d+)\]', line)
            idxs_m=list(map(int, idxs_m))
            mea_qub=line.split()[1].strip().split("[")[0]
            mea_cla=line.split()[3].strip().split("[")[0]
            measure_qub_name.append(mea_qub)
            measure_clas_name.append(mea_cla)
            measure_idxs.append(idxs_m)

            
        quant_reg=[]
        clas_reg=[]
        for j, reg in enumerate(reg_list):
            if reg=='qreg':
                quant_reg.append(QuantumRegister(*reg_idxs[j], reg_names[j]))
            if reg=='creg':
                clas_reg=ClassicalRegister(*reg_idxs[j], reg_names[j])
        circuit_built=QuantumCircuit(*quant_reg, clas_reg)
        for i, elm in enumerate(gate_list):
            if elm == 'x':
                circuit_built.x(*gate_idxs[i])
            elif elm == 'y':
                circuit_built.y(*gate_idxs[i])
            elif elm == 'z':
                circuit_built.z(*gate_idxs[i])
            elif elm == 'h':
                circuit_built.h(*gate_idxs[i])
            elif elm == 's':
                circuit_built.s(*gate_idxs[i])
            elif elm == 't':
                circuit_built.t(*gate_idxs[i])
            elif elm == 'sdg':
                circuit_built.sdg(*gate_idxs[i])
            elif elm == 'tdg':
                circuit_built.tdg(*gate_idxs[i])
            elif elm == 'cx':
                circuit_built.cx(*gate_idxs[i])
            elif elm == 'ccx':
                circuit_built.ccx(*gate_idxs[i])
            elif elm == 'swap':
                circuit_built.swap(*gate_idxs[i])
            elif elm == 'cswap':
                circuit_built.cswap(*gate_idxs[i])
            elif elm[:2] == 'rx':
                phase_str=re.findall('\((.*?)\)', elm)
                if "pi" in elm:
                    reduced_str = phase_str.replace('pi', '1')
                    multiplier = eval(reduced_str)
                    phase=multiplier*pi
                else:
                    phase=float(*phase_str)
                circuit_built.rx(phase, *gate_idxs[i])
            elif elm[:2] == 'ry':
                phase_str=re.findall('\((.*?)\)', elm)
                if "pi" in elm:
                    reduced_str = phase_str.replace('pi', '1')
                    multiplier = eval(reduced_str)
                    phase=multiplier*pi
                else:
                    phase=float(*phase_str)
                circuit_built.ry(phase, *gate_idxs[i])
            elif elm[:2] == 'rz':
                phase_str=re.findall('\((.*?)\)', elm)
                if "pi" in elm:
                    reduced_str = phase_str.replace('pi', '1')
                    multiplier = eval(reduced_str)
                    phase=multiplier*pi
                else:
                    phase=float(*phase_str)
                circuit_built.rz(phase, *gate_idxs[i])
            elif elm[:2] == 'u2':
                temp_str = 'u2(-pi,-pi)'
                qasm_args = re.findall('\((.*?)\)', temp_str)[0].split(",")
                qiskit_args = []
                for arg in qasm_args:
                    if "pi" in arg:
                        reduced_str = arg.replace('pi', '1')
                        multiplier = eval(reduced_str)
                        phase=multiplier*pi
                    else:
                        phase=float(*phase_str)
                    qiskit_args.append(phase)
                circuit_built.u2(*qiskit_args, *gate_idxs[i])
    return circuit_built

In [4]:
def get_conjugate_circuit(qasm_str, return_decomposition=False):
    print("Warning: This will not work for circuits having classical registers.")
    original_circuit = create_circuit(qasm_str)
    U=UnitaryGate(Operator(original_circuit).transpose().conjugate()).to_matrix() 
    conjugate_circuit=QuantumCircuit(len(original_circuit.qubits))
    conjugate_circuit.unitary(U, original_circuit.qubits)
    if return_decomposition:
        print('''This might take some time. It will return the decomposition of the conjugate circuit,
        but might not be the simplest decomposition.''')
        conjugate_circuit = transpile(conjugate_circuit, basis_gates=['u3','cx', 'x', 'h'])
    return conjugate_circuit

## Circuit Creation Example 1

In [5]:
nt=6
qr=QuantumRegister(nt+1, 'q')
testcircuit = QuantumCircuit(qr)
testcircuit.x(nt)
testcircuit.h(nt)
testcircuit.rx(0.2, 3)
testcircuit.rz(0.2, 0)
testcircuit.cx(0,3)
testcircuit.ccx(1,2,4)
testcircuit.draw()    

┌─────────┐          
q_0: ┤ Rz(0.2) ├───────■──
     └─────────┘       │  
q_1: ─────────────■────┼──
                  │    │  
q_2: ─────────────■────┼──
     ┌─────────┐  │  ┌─┴─┐
q_3: ┤ Rx(0.2) ├──┼──┤ X ├
     └─────────┘┌─┴─┐└───┘
q_4: ───────────┤ X ├─────
                └───┘     
q_5: ─────────────────────
        ┌───┐   ┌───┐     
q_6: ───┤ X ├───┤ H ├─────
        └───┘   └───┘

In [6]:
# simulator=AerSimulator()
# testcircuit=transpile(testcircuit, simulator)
# results=simulator.run(testcircuit).result()
# counts = results.get_counts(testcircuit)
# plot_histogram(counts, title="test circuit counts")

In [7]:
print(testcircuit.qasm())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[7];
x q[6];
h q[6];
rx(0.2) q[3];
rz(0.2) q[0];
cx q[0],q[3];
ccx q[1],q[2],q[4];



In [8]:
example_qasmstr = testcircuit.qasm()

In [9]:
print("The original circuit is:")

testcircuit.draw()

The original circuit is:


┌─────────┐          
q_0: ┤ Rz(0.2) ├───────■──
     └─────────┘       │  
q_1: ─────────────■────┼──
                  │    │  
q_2: ─────────────■────┼──
     ┌─────────┐  │  ┌─┴─┐
q_3: ┤ Rx(0.2) ├──┼──┤ X ├
     └─────────┘┌─┴─┐└───┘
q_4: ───────────┤ X ├─────
                └───┘     
q_5: ─────────────────────
        ┌───┐   ┌───┐     
q_6: ───┤ X ├───┤ H ├─────
        └───┘   └───┘

In [10]:
print("The circuit obtained from my function is:")
create_circuit(example_qasmstr).draw()

The circuit obtained from my function is:


┌─────────┐          
q_0: ┤ Rz(0.2) ├───────■──
     └─────────┘       │  
q_1: ─────────────■────┼──
                  │    │  
q_2: ─────────────■────┼──
     ┌─────────┐  │  ┌─┴─┐
q_3: ┤ Rx(0.2) ├──┼──┤ X ├
     └─────────┘┌─┴─┐└───┘
q_4: ───────────┤ X ├─────
                └───┘     
q_5: ─────────────────────
        ┌───┐   ┌───┐     
q_6: ───┤ X ├───┤ H ├─────
        └───┘   └───┘

As we see, they're the same.

Let's now find the conjugate

In [41]:
original_matrix = Operator(create_circuit(example_qasmstr))

In [44]:
conj_matrix = Operator(get_conjugate_circuit(example_qasmstr))

In [45]:
conj_matrix

Operator([[0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
          [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2, 2), output_dims=(2, 2, 2))

Let's multiply the two matrices.

In [47]:
original_matrix*conj_matrix

Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]],
         input_dims=(2, 2, 2), output_dims=(2, 2, 2))

We see that it's an identity matrix

One last thing, let's see the diagram of the conjugate circuit

In [49]:
get_conjugate_circuit(example_qasmstr).draw()

┌──────────┐
q_0: ┤0         ├
     │          │
q_1: ┤1 Unitary ├
     │          │
q_2: ┤2         ├
     └──────────┘

We can decompose this using "return_decomposition=True", but, it's not recommended as it takes a lot of time, and is not the most ideal solution.

In [50]:
# get_conjugate_circuit(example_qasmstr,return_decomposition=False).draw()

## Circuit Creation Example 2

In [51]:
from qiskit.circuit.random import random_circuit

circ = random_circuit(2, 2, seed=520)
circ.draw()

┌───┐┌───┐
q_0: ┤ T ├┤ X ├
     ├───┤└─┬─┘
q_1: ┤ Y ├──■──
     └───┘

In [52]:
print(circ.qasm())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
t q[0];
y q[1];
cx q[1],q[0];



In [53]:
example_qasmstr2 = circ.qasm()

In [54]:
print("The original circuit is:")

circ.draw()

The original circuit is:


┌───┐┌───┐
q_0: ┤ T ├┤ X ├
     ├───┤└─┬─┘
q_1: ┤ Y ├──■──
     └───┘

In [55]:
print("The circuit obtained from my function is:")
create_circuit(example_qasmstr2).draw()

The circuit obtained from my function is:


┌───┐┌───┐
q_0: ┤ T ├┤ X ├
     ├───┤└─┬─┘
q_1: ┤ Y ├──■──
     └───┘

As we see, they're the same.

Let's now find the conjugate

In [56]:
original_matrix = Operator(create_circuit(example_qasmstr2))

In [57]:
conj_matrix = Operator(get_conjugate_circuit(example_qasmstr2))

In [58]:
conj_matrix

Operator([[ 0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        -1.j        ],
          [ 0.        +0.j        ,  0.        +0.j        ,
           -0.70710678-0.70710678j,  0.        +0.j        ],
          [ 0.        +1.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ],
          [ 0.        +0.j        ,  0.70710678+0.70710678j,
            0.        +0.j        ,  0.        +0.j        ]],
         input_dims=(2, 2), output_dims=(2, 2))

Let's multiply the two matrices.

In [59]:
original_matrix*conj_matrix

Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))

We see that it's an identity matrix

One last thing, let's see the diagram of the conjugate circuit

In [60]:
get_conjugate_circuit(example_qasmstr).draw()

┌──────────┐
q_0: ┤0         ├
     │          │
q_1: ┤1 Unitary ├
     │          │
q_2: ┤2         ├
     └──────────┘

We can decompose this using "return_decomposition=True", but, it's not recommended as it takes a lot of time, and is not the most ideal solution.

In [61]:
# get_conjugate_circuit(example_qasmstr,return_decomposition=False).draw()

---

Some things for further improvement:
1. The above method does not work if there are classical registers in the circuit, since we are essentially converting the entire circuit into an operator, finding its matrix representation, taking its conjugate and then defining a unitary gate with this complex conjugate as the underlying matrix. To find the conjugate of a circuit with classical registers, one can try applying the conjugates of the gates in reverse order. This may not be the most efficient solution but it will get the work done.
Similarly, circuits with measurements performed will not get converted to a conjugate
